In [11]:
import argparse
from torch.utils.data import DataLoader
from pre_processing import HandScanDataset2, transform, validation_transform, train_df, valid_df
from main_runner import train


ModuleNotFoundError: No module named 'main_runner'

In [10]:


def main():
    parser = argparse.ArgumentParser(description="Disruptive Auto Encoder Training")
    parser.add_argument('--batch_size', type=int, default=4, help='Batch size for training')
    parser.add_argument('--sw_batch_size', type=int, default=1, help='Sliding window batch size')
    parser.add_argument('--mask_ratio', type=float, default=0.6, help='Mask ratio')
    parser.add_argument('--epoch', type=int, default=300, help='Number of epochs')
    parser.add_argument('--img_size', type=int, default=96, help='Image size')
    parser.add_argument('--min_lr', type=float, default=1e-5, help='Minimum learning rate')
    parser.add_argument('--warmup_epoch', type=int, default=4, help='Warmup epochs')
    parser.add_argument('--base_lr', type=float, default=1e-4, help='Base learning rate')
    parser.add_argument('--warmup_lr', type=float, default=1e-6, help='Warmup learning rate')
    parser.add_argument('--weight_decay', type=float, default=0.05, help='Weight decay')
    parser.add_argument('--cache_dataset', action='store_true', help='Cache dataset')
    parser.add_argument('--cache_rate', type=int, default=1, help='Cache rate')
    parser.add_argument('--model_type', type=str, default='swin_skip', help='Model type')
    parser.add_argument('--save_freq', type=int, default=5, help='Save frequency')
    parser.add_argument('--print_freq', type=int, default=1, help='Print frequency')
    parser.add_argument('--log_dir', type=str, default='./logdir/swin_denoise_mm', help='Log directory')
    parser.add_argument('--output', type=str, default='./output/swin_denoise_mm', help='Output directory')
    parser.add_argument('--thread_loader', action='store_true', help='Use threaded loader')
    parser.add_argument('--out_channels', type=int, default=1, help='Output channels')
    parser.add_argument('--choice', type=str, default='denoise', help='Training choice')
    parser.add_argument('--variance', type=float, default=0.1, help='Variance')
    parser.add_argument('--mm_con', type=float, default=0.03, help='MM contrast ratio')
    parser.add_argument('--temperature', type=float, default=0.5, help='Temperature')
    parser.add_argument('--amp_opt_level', type=str, default='O0', help='AMP optimization level')

    args = parser.parse_args()

    # Set up data loaders
    training_data_dir = 'path/to/your/data'
    train_dataset = HandScanDataset2(labels_df=train_df, data_dir=training_data_dir, transform=transform)
    valid_dataset = HandScanDataset2(labels_df=valid_df, data_dir=training_data_dir, transform=validation_transform)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False)

    # Call the training function from the main_runner.py
    train(args, train_loader, valid_loader)

if __name__ == "__main__":
    main()



usage: ipykernel_launcher.py [-h] --data_dir DATA_DIR
                             [--batch_size BATCH_SIZE]
ipykernel_launcher.py: error: the following arguments are required: --data_dir


SystemExit: 2

/Users/eleanorbolton/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
